# 開発者向けJupyter Notebook
## 概要
このノートブックは開発者が新規機能の実装や機能修正をする際に変更を共有するために使用します。

In [5]:
import os
import sys
import copy
import datetime
import time
import math
import itertools
import unicodedata
import collections
import pickle
import shutil

import numpy as np
import pandas as pd
from PIL import Image
from IPython.display import display, HTML
import matplotlib.pyplot as plt

sys.path.append("../python")
from pyzzle import Puzzle, Dictionary, Placeable, ObjectiveFunction, Optimizer
%load_ext autoreload
%autoreload 2

## フォント設定
本ライブラリにおける画像化には`matplotlib`が用いられますが、`matplotlib`はデフォルトで日本語に対応したフォントを使わないので、`rcParams`を用いてデフォルトのフォント設定を変更します。

In [16]:
# font setting
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 'Takao', 'IPAexGothic', 'IPAPGothic', 'Noto Sans CJK JP']

## 実行

In [7]:
fpath = "../dict/typhoon.txt"  # countries hokkaido animals kotowaza birds dinosaurs fishes sports pokemon typhoon
width = 15
height = 15
seed = 1
withweight = False

np.random.seed(seed=seed)
start = time.time()

In [8]:
# Make instances
puzzle = Puzzle(width, height)
dic = Dictionary(fpath)
objFunc = ObjectiveFunction()
optimizer = Optimizer()

puzzle.importDict(dic)

Puzzle object has made.
 - title       : Criss Cross
 - width       : 15
 - height      : 15
 - cell' shape : (width, height) = (15,15)
Dictionary object has made.
 - file path         : ../dict/typhoon.txt
 - dictionary size   : 140
 - top of dictionary : {'word': 'Damrey', 'weight': 0, 'len': 6}
ObjectiveFunction object has made.
Optimizer object has made.
Imported Dictionary name: `typhoon`, size: 140
Placeable size : 42060


In [9]:
# Register and set method and compile
objFunc.register(["totalWeight", "solSize", "crossCount", "fillCount", "maxConnectedEmpties"])
optimizer.setMethod("localSearch")
puzzle.compile(objFunc=objFunc, optimizer=optimizer)

 - 'totalWeight' function has registered.
 - 'solSize' function has registered.
 - 'crossCount' function has registered.
 - 'fillCount' function has registered.
 - 'maxConnectedEmpties' function has registered.
 - 'localSearch' method has registered.
compile succeeded.
 --- objective functions:
  |-> 0 totalWeight
  |-> 1 solSize
  |-> 2 crossCount
  |-> 3 fillCount
  |-> 4 maxConnectedEmpties
 --- optimizer: localSearch


In [10]:
# Solve
puzzle.firstSolve()

In [11]:
puzzle.solve(epoch=10)
print(f"SimpleSolution: {puzzle.isSimpleSol()}")
puzzle.saveAnswerImage(f"fig/puzzle/{dic.name}_w{width}_h{height}_r{seed}.png")

>>> Interim solution


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,,,,,,,,,,,,,,,
1,,,,,,,,,P,,,M,,,
2,,,,,,,,S,a,o,l,a,,K,
3,,,,,,,,,b,,,t,,o,
4,,,,,,,N,o,u,l,,m,,g,
5,,,,Y,,B,,,k,,N,o,r,u,
6,,,K,u,l,a,p,,,,y,,,m,
7,,,,t,,r,,,C,,a,,L,a,n
8,,,,u,,i,,,h,,t,,,,
9,,,D,,,j,,M,a,l,o,u,,,


>>> Epoch 1/10
    - Improved: [  0  16  15  72 149] --> [  0  22  21  96 164]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,,,B,a,n,y,a,n,,,,,,,
1,,,o,,,,,,P,,,M,,,
2,,,l,,V,,,S,a,o,l,a,,K,
3,B,u,a,l,o,i,,,b,,,t,,o,
4,a,,v,,n,,N,o,u,l,,m,,g,
5,v,,e,,g,,u,,k,,N,o,r,u,
6,i,,n,,f,,r,,,,y,,,m,
7,,,,G,o,n,i,,C,,a,,L,a,n
8,,,,,n,,,,h,,t,,,,
9,,U,s,a,g,i,,M,a,l,o,u,,,


>>> Epoch 2/10
    - Replaced(same score): [  0  22  21  96 164] -> [  0  22  21  96 164]


...

>>> Epoch 4/10
    - Stayed: [  0  22  21  96 164]
>>> Epoch 5/10
    - Stayed: [  0  22  21  96 164]
>>> Epoch 6/10
    - Stayed: [  0  22  21  96 164]
>>> Epoch 7/10
    - Replaced(same score): [  0  22  21  96 164] -> [  0  22  21  96 164]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,,,B,a,n,y,a,n,,,,,,,
1,,,o,,,,,,P,,,M,u,n,
2,,,l,,V,,,S,a,o,l,a,,,
3,B,u,a,l,o,i,,,b,,,t,,,
4,a,,v,,n,,N,o,u,l,,m,,W,
5,v,,e,,g,,u,,k,,N,o,r,u,
6,i,,n,,f,,r,,,,y,,,k,
7,,,,G,o,n,i,,C,,a,,,o,
8,,,,,n,,,,h,,t,,,n,
9,,U,s,a,g,i,,M,a,l,o,u,,g,


>>> Epoch 8/10
    - Stayed: [  0  22  21  96 164]
>>> Epoch 9/10
    - Improved: [  0  22  21  96 164] --> [  0  24  23  99 168]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,,,,,,,,,,,,,,,
1,,,,A,,,,,P,,,M,u,n,
2,,,E,t,a,u,,S,a,o,l,a,,,
3,,,,s,,,,,b,,,t,,,
4,,,,a,,,N,o,u,l,,m,,W,
5,,M,a,n,-,y,i,,k,,N,o,r,u,
6,,e,,i,,,d,,,,y,,,k,
7,,r,,,,,a,,C,,a,,,o,
8,,b,,,,Y,,,h,,t,,,n,
9,K,o,g,u,m,a,,M,a,l,o,u,,g,


>>> Epoch 10/10
    - Stayed: [  0  24  23  99 168]
 --- done
SimpleSolution: True


In [12]:
e_time = time.time() - start
print (f"e_time: {format(e_time)} s")

e_time: 27.711612939834595 s


## Package更新
Jupytextによるpyファイル生成との合わせ技です。  
まずは先にノートブックを保存してから、次のセルを実行してください。  
`../python/pyzzle`ディレクトリのパッケージ情報が更新されます。

In [ ]:
# import os, shutil
# !python ../python/script/ipynbpy2py.py core.py -n pyzzle
# if os.path.exists("../python/pyzzle") is True:
#     shutil.rmtree('../python/pyzzle')
# shutil.move("pyzzle", "../python")

## Sphinxドキュメントを更新
`conda install sphinx`と`pip install sphinx_rtd_theme`で必要ライブラリをインストールしてから次のセルを実行してください。

In [ ]:
###開発中### nbsphinxの利用を検討中
# os.chdir('../doc')
# !sphinx-build -b html ./source ./source/_build
# !sphinx-build -b html ./source ./source/_build #なぜか二度実行するとうまくいく
# os.chdir('../jupyter')